# Import Librares

In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sn
from azureml.core import Workspace, Dataset

# Note 
### In this case I have done the following in a separate notebook and I have the transformed 85k rows data ready
- Data Exploratory Analysis
- Converting categores to Numbers
- Repalcing missing data
 - Feature Engineering
- Split large 450k rows data to 5 parts of 85k each

In [17]:
df = pd.read_csv("Dataset/splitData1_kfold_TrainAndValid.csv")
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Stick_Length_is_missing,Thumb_is_missing,Pattern_Changer_is_missing,Grouser_Type_is_missing,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing,kfold
0,1139248,57000.0,117657,77,121,3.0,1996,4640.0,2,1745,...,True,True,True,True,True,True,True,False,False,3
1,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,1,3716,...,True,True,True,True,True,True,True,True,True,0
2,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,3,4261,...,True,True,True,True,True,True,True,True,True,2
3,1139272,21500.0,1036251,36003,121,3.0,2008,302.0,2,788,...,False,False,False,False,True,True,True,True,True,1
4,1139290,28000.0,1058450,5162,121,3.0,2004,320.0,2,277,...,True,True,True,True,True,True,True,True,True,0


In [18]:
df.shape

(82540, 104)

# Register dataset to the workspace

In [19]:
subscription_id = '292890d4-aa6d-4d5e-a085-97c80db3c30a'
resource_group = 'MLOpsRG'
workspace_name = 'MLOpsWS'

In [20]:
workspace = Workspace(subscription_id, resource_group, workspace_name)
print(workspace)

Workspace.create(name='MLOpsWS', subscription_id='292890d4-aa6d-4d5e-a085-97c80db3c30a', resource_group='MLOpsRG')


In [22]:
# get the datastore to upload prepared data
datastore = workspace.get_default_datastore()
print(datastore)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-31f4ef02-a30d-4a01-993f-1fb82d100b21",
  "account_name": "mlopssa1010",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [23]:
# https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/data/
Dataset.File.upload_directory(src_dir="Dataset", target=(datastore, "data"))

Validating arguments.
Arguments validated.
Uploading file to data
Uploading an estimated of 3 files
Target already exists. Skipping upload for data/weather_dataset_processed.csv
Target already exists. Skipping upload for data/weather_dataset_raw.csv
Uploading Dataset/splitData1_kfold_TrainAndValid.csv
Uploaded Dataset/splitData1_kfold_TrainAndValid.csv, 1 files out of an estimated total of 3
Uploaded 1 files
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

In [24]:
# Retreive uploaded data
dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/splitData1_kfold_TrainAndValid.csv'))

In [25]:
dataset

{
  "source": [
    "('workspaceblobstore', 'data/splitData1_kfold_TrainAndValid.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [26]:
# preview the first 3 rows of the dataset from datastore
dataset.take(3).to_pandas_dataframe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Stick_Length_is_missing,Thumb_is_missing,Pattern_Changer_is_missing,Grouser_Type_is_missing,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing,kfold
0,1139248,57000.0,117657,77,121,3.0,1996,4640.0,2,1745,...,True,True,True,True,True,True,True,False,False,3
1,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,1,3716,...,True,True,True,True,True,True,True,True,True,0
2,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,3,4261,...,True,True,True,True,True,True,True,True,True,2


In [27]:
# Register Dataset to workspace
splitData1_ds = dataset.register(workspace=workspace,
                                 name='splitData1_bulldozers',
                                 description='Supervised Regression data')

In [28]:
print(splitData1_ds)

TabularDataset
{
  "source": [
    "('workspaceblobstore', 'data/splitData1_kfold_TrainAndValid.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "8f0e86b8-a611-4ddb-9f8f-a5c39ad64ec2",
    "name": "splitData1_bulldozers",
    "version": 1,
    "description": "Supervised Regression data",
    "workspace": "Workspace.create(name='MLOpsWS', subscription_id='292890d4-aa6d-4d5e-a085-97c80db3c30a', resource_group='MLOpsRG')"
  }
}
